In [1]:
import glob
import itertools
from os.path import join, splitext, basename
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sk
from sklearn.model_selection import train_test_split
%matplotlib inline

In [2]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import SGD

Using Theano backend.


In [3]:
import trading as trd

In [28]:
# relaod module after modifying it
import importlib
importlib.reload(trd.portfolio)
importlib.reload(trd.benchmarks)
importlib.reload(trd)

<module 'trading' from 'C:\\Users\\hamza\\Documents\\school\\cs_229\\cs229_final_project\\src\\trading\\__init__.py'>

## Load Data

In [4]:
STOCK_DATA_LOC = join('..', 'data', 'stocks')
HI_BETA_DIR = join(STOCK_DATA_LOC, 'high_beta')
LO_BETA_DIR = join(STOCK_DATA_LOC, 'low_beta')

# all the csv's stored in the respective folders
lo_beta_files = glob.glob(join(LO_BETA_DIR, '*.csv'))
hi_beta_files = glob.glob(join(HI_BETA_DIR, '*.csv'))

In [5]:
class StockPair:
    def __init__(self, lo, hi, hist_lo, hist_hi):
        # stock ticker symbols (strings)
        self.lo = hi
        self.hi = lo
        # stock history (pandas data arrays)
        self.hist_lo = hist_lo
        self.hist_hi = hist_hi

In [6]:
# generate all possible combinations of low- and high-beta stocks
stock_pairs = []

for pair in itertools.product(lo_beta_files, hi_beta_files):
    # the stock ticker names
    L, H =  map(lambda f: splitext(basename(f))[0], pair)
    # the actual histories
    hist_l, hist_h = map(lambda f: trd.read_stock_history(f), pair)
    stock_pairs.append(StockPair(L, H, hist_l, hist_h))

train, test = train_test_split(stock_pairs, test_size=5)

## Formulate Problem

In [12]:
Price = np.float64
Shares = np.uint

Action = np.float64
# percent to sell off of the lo-beta stock and buy of hi-beta
actions = np.array([-0.5, -0.3, -0.1, 0, 0.1, 0.3, 0.5])
k = actions.size

In [6]:
m = int(1E3);
n = int(10);
k = int(3);

data = np.random.randa(m, n)
#y = np.round(np.abs((k-1) * np.cos(np.sum(data, 1))))
y = np.random.randint(0, 16, (m, k))

In [7]:
model = Sequential([
    Dense(output_dim=64, input_dim=n, init='uniform'),
    Activation("relu"),
    Dense(output_dim=k)]) #Activation("softmax")

In [8]:
# momentum in [0.5, 0.9, 0.95, 0.99]
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.5, nesterov=True)
model.compile(loss='mean_squared_error', optimizer=sgd, metrics=['accuracy'])

In [9]:
model.fit(data, y, nb_epoch=10, batch_size=32)

Epoch 1/10
1000/1000 [==============================] - 0s - loss: 31.0667 - acc: 0.3550     
Epoch 2/10
1000/1000 [==============================] - 0s - loss: 22.0609 - acc: 0.3470     
Epoch 3/10
1000/1000 [==============================] - 0s - loss: 22.2079 - acc: 0.3350     
Epoch 4/10
1000/1000 [==============================] - 0s - loss: 21.9613 - acc: 0.3360     
Epoch 5/10
1000/1000 [==============================] - 0s - loss: 21.9638 - acc: 0.3250     
Epoch 6/10
1000/1000 [==============================] - 0s - loss: 21.8397 - acc: 0.3230     
Epoch 7/10
1000/1000 [==============================] - 0s - loss: 21.7023 - acc: 0.3810     
Epoch 8/10
1000/1000 [==============================] - 0s - loss: 21.6407 - acc: 0.3670     
Epoch 9/10
1000/1000 [==============================] - 0s - loss: 21.6806 - acc: 0.3440     
Epoch 10/10
1000/1000 [==============================] - 0s - loss: 21.6966 - acc: 0.3340     


In [12]:
model.predict(np.random.rand(2, n))

array([[ 7.62245607,  7.38628244,  7.45234728],
       [ 8.50828552,  8.19099331,  8.26092625]], dtype=float32)